In [324]:
import pandas as pd
import numpy as np
import import_ipynb
import utils_cdt as cdt
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from sklearn import metrics

import matplotlib.pyplot as plt
import time
import pickle
from pyspark.sql import SparkSession, Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel 

In [40]:
infoDataBases = {"ip":"","base":""}

In [268]:
emissor = infoDataBases["base"]
consulta = cdt.resultQuery("ControleVencimentos.sql")
df_vencimentos = cdt.extrairDados(infoDataBases,consulta,
                                            path="../data/",
                                            arquivo=f"controlevencimentos",
                                            spark=False,objSpark=None)

In [42]:
def cache(self): 
    """ 
    Persist this RDD with the default storage level (C{MEMORY_ONLY_SER}). 
    """ 
    self.is_cached = True 
    self.persist(StorageLevel.MEMORY_ONLY_SER) 
    return self 
spark = SparkSession \
    .builder \
    .appName("Modelo - Spark") \
    .config("spark.executor.memory", "4gb") \
    .config("spark.driver.host","127.0.0.1") \
    .getOrCreate()

sc = spark.sparkContext
sc = SQLContext(sc)

In [287]:
emissor = infoDataBases["base"]
consulta = cdt.resultQuery("ControleVencimentos.sql")
df_vencimentos = cdt.extrairDados(infoDataBases,consulta,
                                            path="../data/",
                                            arquivo=f"controlevencimentos",
                                            spark=True,objSpark=spark)

In [337]:
arquivo = spark.read.format("parquet").load("../data/controlevencimentos.parquet")

In [338]:
arquivo.where("Id_ControleVencimentos == 2072").show(3)

+----------------------+--------------+-----------------+-------------------+---------+
|Id_ControleVencimentos|DataVencimento|DataPrevistaCorte|DataRealizacaoCorte|  usuario|
+----------------------+--------------+-----------------+-------------------+---------+
|                  2072|    2020-05-01|       2020-04-22|               null|Per210780|
+----------------------+--------------+-----------------+-------------------+---------+



In [45]:
dataatual = datetime.today().strftime('%Y-%m-%d')

In [333]:
dataatual = '2020-04-22'

In [340]:
t = arquivo.where(to_date("DataPrevistaCorte","yyyy-MM-dd") == dataatual).collect()

In [327]:
def rangeVencLake(path, arquivo, vencimento, meses):
    ult2mes_ano = str((vencimento+relativedelta(months=-2)).year)
    ult2mes_mes = str((vencimento+relativedelta(months=-2)).month)
    ult2mes_mes = ult2mes_mes if len(ult2mes_mes) == '1' else '0'+ult2mes_mes
    ult2mes = int(ult2mes_ano + ult2mes_mes)

    venc_ano = str((vencimento).year)
    venc_mes = str((vencimento).month)
    venc_mes = venc_mes if len(venc_mes) == '1' else '0'+venc_mes
    venc = int(venc_ano + venc_mes)
    
    rangefinal = []
    for x in range(ult2mes,venc+1):
        rangefinal.append(f"{path}{str(x)}/{arquivo}")
    
    return rangefinal

In [329]:
path = "/mnt/datalake-pch/Pernambucanas/Transacoes/"
arquivo = "eventosexternos.parquet"
venc = datetime.datetime.strptime(t[0].DataVencimento, "%Y-%m-%d")
rg = rangeVencLake(path,arquivo,venc,-2)

In [346]:
rg

['/mnt/datalake-pch/Pernambucanas/Transacoes/202003/eventosexternos.parquet',
 '/mnt/datalake-pch/Pernambucanas/Transacoes/202004/eventosexternos.parquet',
 '/mnt/datalake-pch/Pernambucanas/Transacoes/202005/eventosexternos.parquet']

In [118]:
t[0].DataVencimento - datetime.timedelta(month=1)

TypeError: 'month' is an invalid keyword argument for __new__()